In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import pyMELTScalc as M

sys.path.append(r'G:\My Drive\2022\GitHub\pyMELTScalc\MELTS')

import time
import random

In [2]:
M.__version__

'0.0.7dev2'

In [3]:
Data = pd.read_excel('AzoresDataset\Azores.xlsx')
Loc = Data['LOCATION'].str.split(pat = "/")

In [4]:
Azores = pd.DataFrame(np.zeros((len(Data['LOCATION']), 11)), columns = ['Location', 'Stage', 'SiO2', 'TiO2', 'Al2O3', 
                                                                     'FeOt', 'MgO', 'CaO', 'Na2O', 'K2O', 'P2O5'])

for i in range(len(Azores['SiO2'])):
    Azores['Location'].loc[i] = Loc[i][1]
    if len(Loc[i])>2:
        Azores['Stage'].loc[i] = Loc[i][-1]
    
for c in Azores.columns[2:]:
    Azores[c] = Data[c] 

C:\Users\mattg\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
Azores['Location'] = Azores['Location'].str.rstrip()
Loc = ' SAO MIGUEL'

In [6]:
SM = Azores.copy()
SM = SM[SM['Location'] == Loc]

Elements = ['SiO2', 'TiO2', 'Al2O3', 'FeOt', 'MgO', 'CaO', 'Na2O', 'K2O', 'P2O5']
Comp = SM[Elements][(SM['SiO2'] > 45) & (SM['SiO2']< 55)].mean()

Comp = Comp.to_dict()
Comp['MnO'] = 0.0
Comp['CO2'] = 0.0
Comp['H2O'] = 1.0
Comp = {str(key) + '_Liq': val for key, val in Comp.items()}
#Comp['Fe3Fet'] = 0.15

In [ ]:
# example of performing a single calculation
P = 1000
Results = M.multi_iso_crystallise(Model = "MELTSv1.2.0", comp = Comp, Frac_solid = True, Frac_fluid = True, 
                         T_start_C = 1200, T_end_C = 750, dt_C = 5, P_path_bar = P, Fe3Fet_Liq = 0.15, find_liquidus = True)

Running MELTSv1.2.0 calculation...

In [ ]:
M.harker(Results = Results, x_axis = "MgO", y_axis = ["SiO2", "TiO2", "Al2O3", "FeOt", "CaO"], phase = "liquid1", data = SM)

In [ ]:
P = np.linspace(1000, 8000, 16)

In [ ]:
Core = np.array([16, 12, 8, 6, 4, 2, 1])
Time = np.zeros(len(Core))
for i in range(len(Core)):
    Start = time.time()
    Results = M.multi_iso_crystallise(cores = Core[i], Model = "MELTSv1.0.2", comp = Comp, Frac_solid = True, Frac_fluid = True, 
                             T_start_C = 1200, T_end_C = 750, dt_C = 2, P_path_bar = P, find_liquidus = True)
    Time[i] = time.time() - Start

    f, a = plt.subplots(1,1)
    a.plot(Azores['SiO2'][Azores['Location'] == Loc], Azores['Na2O'][Azores['Location'] == Loc] + 
          Azores['K2O'][Azores['Location'] == Loc], 'o', label = Loc)
    
    a.set_title('Processes in Parallel = ' + str(Core[i]))
    a.set_xlabel('SiO$_{2}$ (wt%)')
    a.set_ylabel('Na$_{2}$O + K$_{2}$O (wt%)')

    for R in Results:
        a.plot(Results[R]['All']['SiO2_Liq'], Results[R]['All']['Na2O_Liq'] + Results[R]['All']['K2O_Liq'], '-k')

In [ ]:
f, a = plt.subplots(2,1, figsize = (5, 8))
a[0].plot(Core, Time, 'ok', markerfacecolor = 'red')
a[0].set_ylabel('Time (s)')

a[1].plot(Core, 100*Time/Time[-1], 'ok', markerfacecolor = 'red')
a[1].set_ylabel('% Time (rel. to processes = 1)')
a[1].set_xlabel('Number of Parallel Processes')

In [ ]:
Core = np.array([8, 6, 4, 2, 1])
Time = np.zeros(len(Core))
for i in range(len(Core)):
    Start = time.time()
    Results = M.multi_iso_crystallise(cores = Core[i], Model = "MELTSv1.0.2", comp = Comp, Frac_solid = True, Frac_fluid = True, 
                             T_start_C = 1200, T_end_C = 750, dt_C = 2, P_bar = P, find_liquidus = True)
    Time[i] = time.time() - Start

    f, a = plt.subplots(1,1)
    a.plot(Azores['SiO2'][Azores['Location'] == Loc], Azores['Na2O'][Azores['Location'] == Loc] + 
          Azores['K2O'][Azores['Location'] == Loc], 'o', label = Loc)
    
    a.set_title('Processes in Parallel = ' + str(Core[i]))
    a.set_xlabel('SiO$_{2}$ (wt%)')
    a.set_ylabel('Na$_{2}$O + K$_{2}$O (wt%)')

    for R in Results:
        a.plot(Results[R]['All']['SiO2_Liq'], Results[R]['All']['Na2O_Liq'] + Results[R]['All']['K2O_Liq'], '-k')

In [ ]:
f, a = plt.subplots(2,1, figsize = (5, 8))
a[0].plot(Core, Time, 'ok', markerfacecolor = 'red')
a[0].set_ylabel('Time (s)')

a[1].plot(Core, 100*Time/Time[-1], 'ok', markerfacecolor = 'red')
a[1].set_ylabel('% Time (rel. to processes = 1)')
a[1].set_xlabel('Number of Parallel Processes')